In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
print(torch.cuda.is_available())

True


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# folders = glob.glob('/content/drive/MyDrive/ai_project/data_sample200/train/*')
all_img_list = glob.glob('E:\\open\\train\\*\\*')
# root = '/content/drive/MyDrive/ai_project/data_sample200/train'
test_folder = 'E:\\open\\test'

In [6]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'EARLY_STOPPING': 10,
    'SEED': 41,
}

In [7]:
# fix seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [8]:
# data labeling and grouping

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[3])
le = preprocessing.LabelEncoder()
df['label']=le.fit_transform(df['label'])
print("class:", le.classes_, "\ntotal img:", len(df['img_path']))
print(df['label'].value_counts())

class: [] 
total img: 0
Series([], Name: label, dtype: int64)


In [9]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img_array = np.fromfile(img_path,np.uint8)
        image = cv2.imdecode(img_array,cv2.IMREAD_COLOR)
        # image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [10]:
res_transform = A.Compose([
                          A.Resize(300,300),
                          A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                          ToTensorV2()
                          ])

In [11]:
train, val, _,_  = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])
print("train:", len(train), "validation:", len(val))

ValueError: ignored

In [ ]:
# dataset

batch = 32
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, res_transform)
train_loader = DataLoader(train_dataset, batch_size = batch, shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, res_transform)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False, num_workers=0)

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.alexnet(pretrained=True)
        self.backbone.features[0]=nn.Conv2d(3,64,kernel_size=11,stride=4,padding=2)
        self.backbone.classifier[6]=nn.Linear(4096,num_classes)
        
    def forward(self, x):
        x = self.backbone.features(x)
        x = x.view(-1,256*6*6)
        x = self.backbone.classifier(x)
        return x

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    es_count = 0
    best_epoch = 0
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.long().to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        es_count += 1
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            print(f"Epoch [{epoch}] new best score")
            es_count = 0
            best_score = _val_score
            best_model = model
            best_epoch = epoch
        if es_count == CFG['EARLY_STOPPING']:
            print(f"EARLY STOPPING COUNT: {CFG['EARLY_STOPPING']} BEST EPOCH: {best_epoch}")
            break
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}], es count: {es_count}')
        print("------------------------------------------------------------------------------------------------------------------------------")
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            img=imgs
            imgs = imgs.float().to(device)
            labels = labels.long().to(device)
            
            pred = model(imgs)
           
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
# model load

model = BaseModel()

In [ ]:
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/51 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
res = pd.read_csv('/content/drive/MyDrive/ai_project/test.csv')
res_dataset = CustomDataset('/content/drive/MyDrive/ai_project/'+res['img_path'].values, None, res_transform)
res_loader = DataLoader(res_dataset, batch_size=48, shuffle=False, num_workers=0)

In [ ]:
def label_decoder(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '곰팡이' if l==2 else
        '꼬임' if l==3 else
        '녹오염' if l==4 else
        '들뜸' if l==5 else
        '면불량' if l==6 else
        '몰딩수정' if l==7 else
        '반점' if l==8 else
        '석고수정' if l==9 else
        '오염' if l==10 else
        '오타공' if l==11 else
        '울음' if l==12 else
        '이음부불량' if l==13 else
        '창틀,문틀수정' if l==14 else
        '터짐' if l==15 else
        '틈새과다' if l==16 else
        '피스' if l==17 else
        '훼손' if l==18 else
        'NaN' for l in label
    ]
    return np.array(new_label)

In [ ]:
def inference(model, res_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(res_loader)):
            imgs = imgs.float().to(device)
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()

    pred_kor=label_decoder(preds)
    
    # preds = le.inverse_transform(preds)
    return pred_kor

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ai_project/test.csv')
test_dataset = CustomDataset('/content/drive/MyDrive/ai_project/'+test['img_path'].values, None, res_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
preds = inference(infer_model, test_loader, device)
preds = [str(x) for x in preds]

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ai_project/sample_submission.csv',encoding='utf-8-sig')
submit['label'] = preds
submit.to_csv('/content/drive/MyDrive/ai_project/baseline_submit_densenet.csv', index=False, encoding='utf-8-sig')

In [ ]:
# time

import datetime as dt
x = dt.datetime.now()
time = f'{x.month}_{x.day}_{x.hour + 9}:{x.minute}:{x.second}'

save weights

In [ ]:
model_scripted = torch.jit.script(model) # TorchScript 형식으로 내보내기
model_scripted.save('C:/a/densenet_aug_ver2_{time}.pt') 

In [ ]:
savePath = f'C:/a/alexnet28_21:38.pt'
torch.save(model.state_dict(), savePath)

In [ ]:
model.state_dict()

OrderedDict([('backbone.features.0.weight',
              tensor([[[[ 5.2602e-02,  5.3932e-02, -4.4170e-02,  ...,  3.0761e-02,
                         -4.3159e-02, -3.9087e-02],
                        [ 4.5880e-02, -3.2825e-03,  2.2836e-02,  ...,  1.9606e-02,
                         -3.2634e-02,  4.5693e-02],
                        [ 3.4480e-02, -4.2945e-02, -4.5867e-02,  ..., -2.8640e-02,
                          9.3036e-03, -9.2843e-03],
                        ...,
                        [-2.2133e-02, -8.5529e-03,  4.5584e-02,  ..., -2.7033e-02,
                          4.1264e-02, -4.7878e-02],
                        [ 3.7875e-02,  2.2256e-02,  4.5219e-02,  ..., -4.5896e-02,
                          2.1727e-03,  1.2039e-02],
                        [-4.6045e-02, -1.8174e-03,  4.3591e-02,  ...,  2.7846e-02,
                         -4.2161e-02,  2.3429e-02]],
              
                       [[-4.5868e-02, -7.8144e-03,  5.2771e-03,  ...,  1.9583e-02,
                  

load weights

In [ ]:
model = BaseModel()
model.load_state_dict(torch.load(savePath))
model.eval()